> # Projet informatique 1A - Thomas Aujoux - Grégoire Brugère
    
\
\
**Bienvenu sur l'application : Find My Friends !!**     
\
Le campus de Palaiseau regroupe de nombreux étudiants ayant des passions et des goûts qui sont parfois totalement différent. Au début de l'année, il est très compliqué de rencontrer des personnes aimant les mêmes activités que nous. Avec la collaboration du BDE nous avons décidé de créer une application qui va permettre de créer des groupes d'amis ayant les mêmes passions pour la rentrée des 1A l'année prochaine. Cette application s'appelle **Find My Friends**.

\
Concrètement, vous avez juste à **noter vos activtés de 0 à 10** (0 pour une activité que l'on déteste et 10 pour une activités que l'on adore). L'application vous renvoie directement les personnes avec lesquelles **vous avez le plus de chances d'aimer les mêmes activités**!

\
Ce projet va se décomposer en trois parties :


*   La première partie où l'on crée l'algorithme de matching.
*   La deuxième partie qui permet de créer l'interface grahpique pour simplifier l'utilisateur.
*   La conclusion avec les difficultées rencontrées, nos erreurs et en quoi l'agorithme est performant.

\
Remarque : les tests unitaires se situent juste après le code qu'ils testent.

---



> # Première partie : algorithme de matching

\
# I) Création de la base de données des "faux utilisateurs"

\
Nous allons dans un permier temps créer la base de donnée des différents utilisateurs. Chaque colonnes va représenter un centre d'intérêt et chaque lignes un utilisateur. 

Etant donné que nous allons dans un deuxième temps réaliser une réduction de dimension de la base de donnée (APC) nous pouvons être complétement exhaustif sur le nombre de données !

Le thème de cette année n'étant pas le scrapping (aller chercher des informations sur internet) nous allons créer nous mêmes les profils des utilisateurs. Pour la créer nous allons procéder en deux étapes

Création de la base de données:

*   Nous allons mettre en places des catégories ("Ecriture","Suchi","Hockey","Théâtre","Sport à la maison" ...) remplies de 0 à 10, les "faux utilisateurs" vont mettre une note de 0 à 10 pour 0 une activité que l'on n'aime pas et 10 une activité que l'on adore faire

*   Ensuite, il y aura des biographies "Bios" qui vont permettre aux "faux utilisateurs" de mettre des passions qui ne sont pas présentent dans la liste 



# I) A) Mise en place des catégories


On crée les catégories :



*   On importe les bibliothèques utilisées, et on définie le nombre de "faux individus" qui seront présents dans notre base de données.

*   On définie les catégories qui vont être les colonnes de notre base de données, comme nous allons par la suite faire une réduction de dimension, on peut en définir autant que l'on veut ! Les catégories choisies sont prises parmis les passions les plus courantes des français.

*   On remplie les différentes colonnes avec des nombres aléatoires de 0 à 10, 0 pour un individu qui n'aime pas du tout cette activité, 10 pour un individu qui adore cette activité.

*   Ensuite, pour réaliser une APC il faut mettre les données à la même échelle.

*   Finalement, nous allons réaliser un test unitaire pour savoir si toutes les données du tableau sont bien comprises entre 0 et 1.



In [1]:
import pandas as pd
import numpy as np

n=10000
#categories = ["Informatique","Sport","Cinéma","mode", "Ecriture", "Suchi", "Hockey", "Théâtre","Sport à la maison","hackaton", "Mangas", "Sneakers", "Maquillage", "Instagram", "Arts martiaux", "Marvel", "Marche à pied", "Course à pied", "voyage", "Discussion dans une autre langue", "Réseaux sociaux", "Cosmétique", "Skateboard", "Cuisine végane", "Photographie", "Lecture", "Chant", "Volleyball", "Sports", "Vinyasa", "Café","League of Legends","Karaoké","Fortnite","Plongée en apnée","Itinéraire gourmand","Statistiques","Mathématiques","Natation","Tennis","Rugby","escalade","bowling","course"]
#Nous avons opter pour une liste de categories simplifié, rendantainsi le matching moins spécifique mais rendant le programme plus simple et plus efficace à moindre échelle
categories = ["Informatique","Sport","Cinéma","Mode","Littérature", "Gastronomie", "Hockey", "Théâtre", "Hackaton"]
len(categories)

9

Nous avons importer les modules nécessaires. De plus nous avons défini le nombre d'individus que nous voulons créer et les catégories que nous proposons à notre application.

In [2]:
def remplir(categorie,nombre_indiv):
  dat = pd.DataFrame(columns = categorie)
  for i in dat.columns:
    dat[i] = np.random.randint(0,10,nombre_indiv)
  return dat

data = remplir(categories,n)
data



,Informatique,Sport,Cinéma,Mode,Littérature,Gastronomie,Hockey,Théâtre,Hackaton
0,4,3,3,4,5,6,2,5,4
1,5,8,0,9,3,7,2,4,6
2,0,4,7,4,7,1,0,4,6
3,2,5,0,0,2,2,8,6,7
4,9,6,5,5,7,3,3,9,6
...,...,...,...,...,...,...,...,...,...
9995,5,9,7,2,3,6,6,8,6
9996,5,1,8,6,5,0,0,4,8
9997,2,7,3,5,1,8,2,8,3
9998,3,7,2,6,8,5,7,4,0


Nous avons créer une fonction qui à partir de plusieurs catégories.Nous avons actuellement 42 catégories et 1000 individus qui ont données leurs opinions.

Comme nous allons faire ensuite un algorithme de clustering, avec autant de donnée cela peut prendre beaucoup de temps à réaliser. Faire une mise à l'échelle de données va permettre de réduire le temps d'exécution pour la suite. Pour cela nous allons utiliser une commande classique de sklearn.

In [6]:
from sklearn.preprocessing import MinMaxScaler

def echelle_fn(dat, categorie):
    dat = pd.DataFrame(MinMaxScaler().fit_transform(dat))
    dat = dat.set_axis(categorie, axis='columns')
    return dat

echelle1=echelle_fn(data,categories)
echelle1

,Informatique,Sport,Cinéma,Mode,Littérature,Gastronomie,Hockey,Théâtre,Hackaton
0,0.444444,0.333333,0.333333,0.444444,0.555556,0.666667,0.222222,0.555556,0.444444
1,0.555556,0.888889,0.000000,1.000000,0.333333,0.777778,0.222222,0.444444,0.666667
2,0.000000,0.444444,0.777778,0.444444,0.777778,0.111111,0.000000,0.444444,0.666667
3,0.222222,0.555556,0.000000,0.000000,0.222222,0.222222,0.888889,0.666667,0.777778
4,1.000000,0.666667,0.555556,0.555556,0.777778,0.333333,0.333333,1.000000,0.666667
...,...,...,...,...,...,...,...,...,...
9995,0.555556,1.000000,0.777778,0.222222,0.333333,0.666667,0.666667,0.888889,0.666667
9996,0.555556,0.111111,0.888889,0.666667,0.555556,0.000000,0.000000,0.444444,0.888889
9997,0.222222,0.777778,0.333333,0.555556,0.111111,0.888889,0.222222,0.888889,0.333333
9998,0.333333,0.777778,0.222222,0.666667,0.888889,0.555556,0.777778,0.444444,0.000000




---

# Teste unitaire 
Le but du test unitaire est de vérifier si les données du tableau sont bien comprises entre 0 et 1. Cela permet de vérifier si notre mise à l'échelle des données a bien fonctionnée.
Comme le nombre d'opérations est excéssivement important, nous allons regarder les 100 premières lignes. Etant donnée que les variables sont générées aléatoirement cela ne pose pas de problème.



In [7]:

def test1_fn(echell):
  test = True 
  for i in range(100):
    for j in echell.columns:
      if echell.iloc[i][j]<0 or echell.iloc[i][j]>1:
        test = False
  return test

test1 = test1_fn(echelle1)
test1


True



---



# I) B) Création des biographies ("Bios")

\
On crée les biographies :

Notre but va être des générer des fausses phrases pour imiter des vraies biographies faites pas des utilisateurs, le but de ses "Bios" est de donner une plus grande possibilité aux utilisateurs de s'exprimer en donnant des mots qui n'était pas présents parmis les catégories. Pour cela nous allons précéder en plusieurs étapes :

* Chaque individus se vera attribuer au hasard un mot d'une très grande liste des mots (la liste a été trouvée sur internet, ce sont les sports commançant par la lettre "A"). 
*   Pour rendre les biographies plus réalistes, nous allons ajouter dans la liste des mots des "fausses activités" ('le', 'la', 'les', 'je', 'j','') pour donner l'impression que se sont des vraies phrases. En effet, dans la réalité il aurait été nécessaire de traiter les bios pour les rendre pluslisibles, le mot "la" ne donnant aucunes informations sur la préférence d'une activité.
*   Nous allons dans un premier temps traiter la liste des biographies en enlevant toutes les majuscules pour les transformer en minuscules, ensuite nous allons remplacer certaines lettres par d'autres ("é" en "e", "è" en "e" ...). Cela va permettre de simplifier les biographies.

*   La deuxième simplification des biographies va consister à sépare tous les mots ayant des espaces entre eux, pour permettre de voir apparaitre des catégories. Ensuite, nous allons enlever tous les mots que nous considérons comme inutiles tels que ('le', 'la', 'les', 'je', 'j','').

*   Les mots étant aléatoires, il est possible que le seul mot présent pour un utilisateur ne soit pas utilisable. Nous allons donc attribuer des mots aléatoires pour ces utilisateur.

*   Finalement, nous allons mettre en forme dans notre base de données les activités.





In [8]:
import random
L = ['le', 'la', 'les', 'je', 'j','',"Brunch","Fripes","Voguing","Couchsurfing","Mèmes","Happy-Hour","Moto","Investissement","Art","Randonnée","Montagnes","Backpachking","Pêche","Tennis","Glace","Pétanque","Patinoire","Expositions","Ski","Snowboard","Pilates","Broadway","Cheerleading","Chorale","Street-food","Accrobranche","Acrosport","Aerobic","Aéromodélisme","Aérostation","Agility","Aikido","Airsoft","Alpinisme","Apnée","Aquabike","Aquagym","Athlétisme","Aviation","Aviron"]

In [9]:
def biographie(L_,nombre_indiv):
  Bios = []
  for i in range(nombre_indiv):
    Bios.append(random.choice(L_))
  return Bios

Bios = biographie(L,n)
for i in range(0,15):
  print(Bios[i])

Randonnée
Aerobic
Aviron
Tennis
Accrobranche
Pêche
Aikido
Glace
Street-food
Chorale
Athlétisme
la
Chorale
Snowboard
Athlétisme


"Bios" est la liste des biographies des utilisateurs. Elle contient de nombreux mots piochée parmis une liste contenant d'autres "catégories" que les utilisateurs auraient pu rentrer eux-mêmes. En plus de ces catégories, il y a des mots de la langue française qu'il va falloir supprimer.

In [10]:
import string as str

mot_inutile = ['le', 'la', 'les', 'je', 'j', '', 'du', 'de','a','']
liste_remplacement = [['.', ' '], ["'", ' '], ['é', 'e'], ['è', 'e'], ['ê', 'e'], [':', '']]

Nous créons une liste de mots que nous allons supprimer des biographies car ils n'expriment pas d'opinion ('le', 'la', 'les', 'je', 'j', '') et certains mots sont remplacés par d'autre pour pouvoir simplifier le traitement après.

In [11]:
def simplification(L_,nombre_indiv,liste_remplace):
  texte_simple=""
  texte_brut = biographie(L_,nombre_indiv) 
  for j in texte_brut:
    texte_simple = texte_simple+ " " + j.lower()
  for i in liste_remplace:
    texte_simple = texte_simple.replace(i[0], i[1])
  return(texte_simple)

simplification(L,n,liste_remplacement)

' les chorale alpinisme alpinisme agility aerobic aquagym la happy-hour expositions broadway aerostation voguing broadway fripes aerostation alpinisme aerostation fripes  aquagym peche aerobic memes aviron memes randonnee happy-hour acrosport les cheerleading happy-hour aerobic aikido patinoire accrobranche aeromodelisme expositions montagnes voguing acrosport peche acrosport cheerleading pilates petanque airsoft randonnee aikido couchsurfing cheerleading montagnes peche alpinisme brunch aerostation happy-hour broadway happy-hour chorale montagnes pilates aviron les chorale patinoire brunch aerobic accrobranche brunch backpachking  aikido alpinisme petanque chorale randonnee art montagnes aquabike petanque aerobic randonnee airsoft chorale aviron airsoft pilates  aeromodelisme randonnee aquagym aeromodelisme investissement randonnee cheerleading art alpinisme montagnes ski memes chorale montagnes investissement randonnee  le peche agility je broadway airsoft athletisme memes exposition

Le but de cette simplification de la biographie est d'enlever les majuscules et de remplacer certains mots par d'autres (les mots sont donnés dans la liste remplacement).

In [12]:
def simplification2(L_,nombre_indiv, liste_remplace,mot_inutil):
    ltexte = (simplification(L_,nombre_indiv, liste_remplace)).split(' ')
    texte_toke = []
    for mot in ltexte:
        if mot not in mot_inutil:
            texte_toke.append(mot)
    return(texte_toke)

In [13]:
liste_mots = simplification2(L,n, liste_remplacement, mot_inutile)

for i in range(0,15):
  print(liste_mots[i])

snowboard
aquagym
patinoire
couchsurfing
apnee
airsoft
fripes
aikido
montagnes
chorale
aerostation
snowboard
alpinisme
tennis
tennis


Evidemment, étant donné que les biographies sont générées avec des mots aléatoires, nous ne sommes pas à l'abris qu'une des biographie soit uniquement générée avec des mots inutiles ('le', 'la', 'les', 'je', 'j',''), que nous avons supprimés précédemment. Il faut donc remplir la biographie avec des catégories.

In [14]:
I = ["Brunch","Fripes","Voguing","Couchsurfing","Mèmes","Happy-Hour","Moto","Investissement","Art","Randonnée","Montagnes","Backpachking","Pêche","Tennis","Glace","Pétanque","Patinoire","Expositions","Ski","Snowboard","Pilates","Broadway","Cheerleading","Chorale","Street-food","Accrobranche","Acrosport","Aerobic","Aéromodélisme","Aérostation","Agility","Aikido","Airsoft","Alpinisme","Apnée","Aquabike","Aquagym","Athlétisme","Aviation","Aviron"]

def biographie_finale(echell,L_,I_,nombre_indiv,liste_remplace,mot_inutil):
  Bios = biographie(L_,nombre_indiv)
  liste_mot = simplification2(L_,nombre_indiv,liste_remplace, mot_inutil)
  for i in range((nombre_indiv-len(liste_mot))):
    liste_mot.append(random.choice(I_))
  echell = echell.assign(Bios= liste_mot)
  return echell

data_nouv = biographie_finale(echelle1,L,I,n,liste_remplacement,mot_inutile)
data_nouv["Bios"]
data_nouv

,Informatique,Sport,Cinéma,Mode,Littérature,Gastronomie,Hockey,Théâtre,Hackaton,Bios
0,0.444444,0.333333,0.333333,0.444444,0.555556,0.666667,0.222222,0.555556,0.444444,aquagym
1,0.555556,0.888889,0.000000,1.000000,0.333333,0.777778,0.222222,0.444444,0.666667,cheerleading
2,0.000000,0.444444,0.777778,0.444444,0.777778,0.111111,0.000000,0.444444,0.666667,chorale
3,0.222222,0.555556,0.000000,0.000000,0.222222,0.222222,0.888889,0.666667,0.777778,aerostation
4,1.000000,0.666667,0.555556,0.555556,0.777778,0.333333,0.333333,1.000000,0.666667,aerostation
...,...,...,...,...,...,...,...,...,...,...
9995,0.555556,1.000000,0.777778,0.222222,0.333333,0.666667,0.666667,0.888889,0.666667,Aéromodélisme
9996,0.555556,0.111111,0.888889,0.666667,0.555556,0.000000,0.000000,0.444444,0.888889,Acrosport
9997,0.222222,0.777778,0.333333,0.555556,0.111111,0.888889,0.222222,0.888889,0.333333,Athlétisme
9998,0.333333,0.777778,0.222222,0.666667,0.888889,0.555556,0.777778,0.444444,0.000000,Airsoft


Avant de traiter la base de données il faut créer des nouvelles catégories avec les biographies et compléter la base de données pour chaque utilisateurs. Cela s'appelle une vectorisation !

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

def bio_vectoriser(echell,L_,I_,nombre_indiv):
  data_nouv = biographie_finale(echell,L_,I_,nombre_indiv,liste_remplacement,mot_inutile)
  vectorizer = CountVectorizer()
  tableau = vectorizer.fit_transform(data_nouv["Bios"])
  nouvelles_bios = pd.DataFrame(tableau.toarray(), columns=vectorizer.get_feature_names_out())
  df = pd.concat([data_nouv, nouvelles_bios], axis=1)
  df.drop('Bios', axis=1, inplace=True)
  return df

df = bio_vectoriser(echelle1,L,I,n)
df_copie = df
df


,Informatique,Sport,Cinéma,Mode,Littérature,Gastronomie,Hockey,Théâtre,Hackaton,accrobranche,...,pilates,pétanque,pêche,randonnee,randonnée,ski,snowboard,street,tennis,voguing
0,0.444444,0.333333,0.333333,0.444444,0.555556,0.666667,0.222222,0.555556,0.444444,0,...,0,0,0,0,0,0,0,0,0,0
1,0.555556,0.888889,0.000000,1.000000,0.333333,0.777778,0.222222,0.444444,0.666667,0,...,1,0,0,0,0,0,0,0,0,0
2,0.000000,0.444444,0.777778,0.444444,0.777778,0.111111,0.000000,0.444444,0.666667,0,...,0,0,0,0,0,0,0,0,0,0
3,0.222222,0.555556,0.000000,0.000000,0.222222,0.222222,0.888889,0.666667,0.777778,0,...,0,0,0,0,0,0,0,0,0,0
4,1.000000,0.666667,0.555556,0.555556,0.777778,0.333333,0.333333,1.000000,0.666667,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.555556,1.000000,0.777778,0.222222,0.333333,0.666667,0.666667,0.888889,0.666667,0,...,0,0,0,0,0,0,0,0,0,0
9996,0.555556,0.111111,0.888889,0.666667,0.555556,0.000000,0.000000,0.444444,0.888889,0,...,0,0,0,0,0,0,0,0,0,0
9997,0.222222,0.777778,0.333333,0.555556,0.111111,0.888889,0.222222,0.888889,0.333333,0,...,0,0,0,0,0,0,0,0,0,0
9998,0.333333,0.777778,0.222222,0.666667,0.888889,0.555556,0.777778,0.444444,0.000000,0,...,0,0,0,0,0,0,0,0,0,0


# II) Réduction de dimension

\
**Idée générale :**

Nous avons à présent un très grand nombre de colonnes. Il est nécessaire de réduire la dimension de notre base de données pour avoir un algorithme plus efficace.
Cette méthode va permettre de conserver une grande partie de l'information statistique tout en réduisant considérablement le temps de l'algorithme.

Le principe est simple : Il s’agit en fait de résumer l’information qui est contenue dans une large base de données en un certain nombre de variables synthétiques appelées : Composantes principales. 

L’idée est ensuite de pouvoir projeter ces données sur l’hyperplan le plus proche afin d’avoir une représentation simple de nos données.
Évidemment, qui dit réduction de dimension dit perte d’informations. C’est là tout l’enjeu que représente une Analyse en Composantes principales. Nous avons choisis de garder 90% de l'information.

\
**Les différentes étapes :**


*   Nous allons dans un premier temps afficher la variance expliquée en fonction du nombre de valeurs propres utilisées. Nous remarquons que souvent à partir d'un certain nombre de valeurs propres rajoutter une autre valeur propre ne permet pas de rajouter énormément d'information.

*   On va réaliser une PCA à partir de ce pourcentage de "variance expliquée" trouvée dans l'étape précédente.





In [19]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

Ce graphique nous permet de comprendre combien de vecteurs propres permettent d'expliquer une proportion de la variance, donc de l'information. Avec un certain nombre de "Components" cela permet d'expliquer un certain pourcentage de la variance.

In [20]:
def pca_fn(df,n):
  pca=PCA(n)
  df_pca = pca.fit_transform (df)
  pca = PCA(n_components=pca.n_components_)
  df_pca = pca.fit_transform(df)
  return df_pca

df_pca = pca_fn(df,0.97)
#df_pca_copy=df_pca
df_pc = pca_fn(df,0.97)

L'ACP nous a permis de réduire le nombre de composantes principales ou de fonctionnalités. Etant donnée que notre projet est basé sur une base de données générée aléatoirement les résultats ne sont pas toujours les mêmes. En revanche, sur le premier graphique on voit que la courbe est de moins en moins pentue et qu'à partir de 0.97 variance explained le fait de rajouter une catégorie ne permet pas de rajouter beaucoup d'information par rapport aux catégories qui se situent avant. Nous avons donc décidé de réaliser notre PCA en conservant 0.97% de variance expliquée. 
Cela nous permet d'enlever un peu près (dépend du fait que la base de données est générée aléatoirement) 15 catégories. 

# III) Création des cluster

\

**Idée générale :**

Maintenant que notre base de donnée est prête nous pouvons réaliser des cluster. Nous avons choisi de réaliser un algorithme de cluster par méthode hiérarchique car nous connaissons exactement le nombre de cluster que nous voulons étant donné que l'on veut des groupes de 5 personnes.

**Les différentes étapes :**

Le principe de la méthode hiérarchique est très simple, elles forment des connexion de pas à pas entre individus et utilise une matrice de distance pour retrouver le regroupement le plus proche d'un autre. 

*   Première étape : repére pour chaque individus l'autre individu qui est le plus proche et forment des groupes de 2 parmis la population.

*   Deuxième étape : il faut mettre à jour la matrice des distance en enlevant une case, à cause du regroupement de deux individus. Ensuite, en fonction de la statégie d'agréagation on repère le groupe le plus proche du premier et ensuite on les fusionne. 

*   On recommence l'algorithme jusqu'à qu'on obtienne le bon nombre de cluster.







In [21]:
from sklearn.cluster import AgglomerativeClustering

def clusteriser(dat,df_pc,nombre_cluste,L_,I_,nombre_indiv):

  cluster = AgglomerativeClustering(n_clusters=nombre_cluste)  
  cluster.fit(df_pc)
  numeros_cluster = cluster.labels_
  biographie_fi = biographie_finale(dat,L_,I_,nombre_indiv,liste_remplacement,mot_inutile)

  biographie_fi['Cluster #'] = numeros_cluster
  return biographie_fi

nombre_cluster = int(n/10)
data_final = clusteriser(data,df_pc, nombre_cluster,L,I,n)
data_final

#biographie_finale(echell,L_,I_,nombre_indiv,liste_remplace,mot_inutil)

,Informatique,Sport,Cinéma,Mode,Littérature,Gastronomie,Hockey,Théâtre,Hackaton,Bios,Cluster #
0,4,3,3,4,5,6,2,5,4,moto,11
1,5,8,0,9,3,7,2,4,6,apnee,174
2,0,4,7,4,7,1,0,4,6,aviation,360
3,2,5,0,0,2,2,8,6,7,aikido,784
4,9,6,5,5,7,3,3,9,6,agility,275
...,...,...,...,...,...,...,...,...,...,...,...
9995,5,9,7,2,3,6,6,8,6,Montagnes,975
9996,5,1,8,6,5,0,0,4,8,Aéromodélisme,785
9997,2,7,3,5,1,8,2,8,3,Chorale,437
9998,3,7,2,6,8,5,7,4,0,Snowboard,131


In [22]:
def trouver_cluster(data_fi,num):
  df_mask=data_fi['Cluster #'] == num
  filtered_df = data_fi[df_mask]
  return(filtered_df)

pd.DataFrame(trouver_cluster(data_final,50))



,Informatique,Sport,Cinéma,Mode,Littérature,Gastronomie,Hockey,Théâtre,Hackaton,Bios,Cluster #
1055,2,7,9,7,6,5,0,4,8,tennis,50
3571,0,7,0,6,7,1,6,5,7,pilates,50
4528,1,9,4,6,9,3,4,6,7,petanque,50
4552,1,9,9,9,9,6,1,2,7,fripes,50
5044,6,9,5,8,6,1,4,2,6,expositions,50
5566,1,8,5,8,7,3,4,3,7,aquagym,50
6137,1,9,4,9,5,0,5,2,5,voguing,50
7351,0,7,1,8,2,1,6,5,5,randonnee,50
8819,3,9,8,8,7,3,2,3,3,Mèmes,50
9471,2,8,1,6,2,1,4,5,6,Aviation,50




---
# Test unitaire

Le but de ce test est de vérifier si deux individus ayant les mêmes caractéristiques vont être assignées dans le même cluster. Nous allons donc créer une copie d'un individu pour la remettre dans la base de données. Le test sera vérifié si les deux individus sont dans le même cluster.



In [23]:
def creer_cluster(categorie,nombre_indiv,indiv,L_,I_,nombre_cluste):
  dat = remplir(categorie,nombre_indiv)
  #nouv_ligne = pd.DataFrame([individu],columns = categorie)
  #dat = pd.DataFrame(dat.concat([dat, nouv_ligne], ignore_index=True))
  dat.loc[nombre_indiv-1]= indiv
  z= echelle_fn(dat,categorie)
  df = bio_vectoriser(z,L_,I_,nombre_indiv)
  df_pca2 = pca_fn(df,0.97)
  data_fin = clusteriser(dat,df_pca2,nombre_cluste,L_,I_,nombre_indiv)
  print(data_fin)
  numero = data_fin["Cluster #"].loc[nombre_indiv-1]
  print(numero)
  return trouver_cluster(data_fin,numero)

In [24]:
a =creer_cluster(categories,n,[5]*9,L,I,nombre_cluster)
a

      Informatique  Sport  Cinéma  Mode  Littérature  Gastronomie  Hockey  \
0                5      4       6     8            5            6       1   
1                9      2       3     1            7            9       4   
2                9      7       9     3            6            4       6   
3                9      4       5     9            8            1       4   
4                7      6       9     4            1            5       3   
...            ...    ...     ...   ...          ...          ...     ...   
9995             0      7       1     3            5            6       0   
9996             7      9       7     1            7            0       9   
9997             8      4       3     1            8            9       9   
9998             0      3       3     4            9            0       2   
9999             5      5       5     5            5            5       5   

      Théâtre  Hackaton         Bios  Cluster #  
0           2         9  

,Informatique,Sport,Cinéma,Mode,Littérature,Gastronomie,Hockey,Théâtre,Hackaton,Bios,Cluster #
583,6,2,6,4,3,7,8,3,6,investissement,498
1363,8,0,1,2,0,5,6,8,6,street-food,498
1412,9,4,4,5,2,6,7,5,5,ski,498
2009,6,4,2,1,2,7,9,1,2,aquagym,498
2933,9,7,2,1,3,6,9,5,7,aerostation,498
3367,8,1,2,2,1,5,9,1,5,airsoft,498
4041,9,6,4,2,0,7,6,3,8,fripes,498
5907,8,7,8,6,3,8,8,6,9,acrosport,498
6015,7,0,6,2,4,9,9,4,1,snowboard,498
6551,8,0,4,2,6,6,7,3,7,aikido,498




---



> # Deuxième partie : Création de l'interface

# I)



In [ ]:
from tkinter import *
from tkinter import messagebox as mb


class Quiz:
	def __init__(self):
		self.qno=0 #Cette acumulateur indique quelle est le numér de la question traitée
		self.disp_title() #Fonction qui va permettre d'afficher le titre de test
		self.disp_ques() #Fonction qui va permettre d'afficher les questions
		self.opt_sel=IntVar() #Variable indiquant quelle réponse a été selectionné
		self.opts=self.radio_buttons() #Cela concerne l'affichage des réponses possibles, qui seront faites sous la forme de "radiobuttons"
		self.disp_opt() #Fonction permettant d'afficher les réponses possibles
		self.buttons() #Cette fonction sert à afficher les boutons de validation des questions et d'arrêt du test
		self.total_size=len(question)
		self.recup= [] #Cette liste servira à récupérer les réponses entrées par l'utilisateur

	def disp_res(self):
		
		result = f"Résultats: {creer_cluster(categories,n,self.recup,L,I,nombre_cluster)}" #à noter qu'afin de rendre le test plus court les autres questions ont été répondu à 0.5
		
		mb.showinfo("Résultat :", f"{result}")
        

	def next_btn(self):
		(self.recup).append(self.opt_sel.get() -1) #On insére la valeur entré par l'utilisateur dans la liste recup
		self.qno += 1
		
		if self.qno==self.total_size: #Cette condition implique que toutes les questions ont été répondu, nous pouvons passé au résultat
			self.disp_res()
			ws.destroy()
		else:
			self.disp_ques()
			self.disp_opt()


	def buttons(self):
		
		next_button = Button(
            ws, 
            text="Suivant",
            command=self.next_btn,
            width=10,
            bg="#F2780C",
            fg="white",
            font=("ariel",16,"bold")
            )
		
		next_button.place(x=350,y=380)
		
		quit_button = Button(
            ws, 
            text="Quit", 
            command=ws.destroy,
            width=5,
            bg="black", 
            fg="white",
            font=("ariel",16," bold")
            )
		
		quit_button.place(x=700,y=50)   #Cette fonction définit globalement le design des boutons "QUIT" et "Suivant" et leurs commandes


	def disp_opt(self):
		val=0
		self.opt_sel.set(0)
		
		for option in options[self.qno]:
			self.opts[val]['text']=option #On parcourt la liste des options afin de toutes les affichées
			val+=1

	def disp_ques(self):
		
		qno = Label(
            ws, 
            text=question[self.qno], 
            width=60,
            font=( 'ariel' ,16, 'bold' ), 
            anchor= 'w',
			wraplength=700,
			justify='center'
            )
		
		qno.place(x=70, y=100)


	def disp_title(self):
		
		title = Label(
            ws, 
            text="Quiz de compatibilité",
            width=50, 
            bg="#F2A30F",
            fg="white", 
            font=("ariel", 20, "bold")
            )
		
		title.place(x=0, y=2)


	def radio_buttons(self):
		
		q_list = []
		
		x_pos = 130
		
		while len(q_list) < 11: #Cette boucle while permet d'afficher les 11 boutons nécessaire à noter les catégories
			
			radio_btn = Radiobutton(
                ws,
                text=" ",
                variable=self.opt_sel,
                value = len(q_list)+1,
                font = ("ariel",18)
                )
			q_list.append(radio_btn)
			
			radio_btn.place(x = x_pos, y = 200)
			
			x_pos += 50
		
		return q_list

ws = Tk()

ws.geometry("800x450")

ws.title("Quiz de compatibilité")
question = ["Votre intérêt pour l'informatique?","Votre intérêt pour le sport","Votre intérêt pour la cinématographie","Votre intérêt par rapport à la mode?","Votre intérêt pour la littérature?","Votre intérêt pour la cuisine japonaise?","Votre intérêt pour le Hockey?","Votre intérêt pour le Théâtre?","Votre intérêt pour les hackatons?"]
options = [['0','1','2','3','4','5','6','7','8','9','10']]*len(question)

quiz = Quiz()

ws.mainloop()